**BERTTWEET TEXT CLASSIFICATION - Immigration**


Alkan Can Mollaibrahimoglu CSS MA Koç University




**Relevant count:** 345

**Irrelevant sample:** 1045

### Importing Necessary Libraries

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

### Loading Relevant and Irrelevant Data Sets

In [ ]:
df=pd.read_excel('/content/final_data_labels.xlsx')

In [ ]:
df=df[['Tweet','label']]
df

,Tweet,label
0,zurnanın zırt dediği yerde tamda burası bugüne...,1
1,kusura bakmada bundan yıl önce çocukken bende ...,1
2,ülkemizin zor günlerinde tapu kadastro teknike...,0
3,türk misafirperverliğini aramaktayım misafire ...,0
4,irkçılık cezalandırılmağı suya karışmış zehir ...,1
...,...,...
15312,mülteciler sadece türkiye’nin değil bütün düny...,1
15313,iğdir vali̇si̇nden i̇bretli̇k uygulama olay çı...,1
15314,mülteci sorunu merkezi hükümeti ilgilendiren u...,1
15315,yaralanan çocukların fotoğraflarını gördüm kor...,1


In [ ]:
df= df.dropna(subset=['Tweet', 'label'])


In [ ]:
df['label'].value_counts()

label
0    14970
1      345
Name: count, dtype: int64

### Sampling Irrelevant Set

In [ ]:
df_relevant=df[df['label']==1]

In [ ]:
df_relevant

,Tweet,label
0,zurnanın zırt dediği yerde tamda burası bugüne...,1
1,kusura bakmada bundan yıl önce çocukken bende ...,1
4,irkçılık cezalandırılmağı suya karışmış zehir ...,1
6,çocuk fabrikası kurdunuz dabizim haberimiz mi ...,1
7,ırkçısınız sığınmacı düşmanısınız faşistsiniz ...,1
...,...,...
15312,mülteciler sadece türkiye’nin değil bütün düny...,1
15313,iğdir vali̇si̇nden i̇bretli̇k uygulama olay çı...,1
15314,mülteci sorunu merkezi hükümeti ilgilendiren u...,1
15315,yaralanan çocukların fotoğraflarını gördüm kor...,1


In [ ]:
df_irrelevant=df[df['label']==0]

In [ ]:
df_irrelevant=df_irrelevant.sample(n=1035,random_state=5)

In [ ]:
df_combined = pd.concat([df_relevant, df_irrelevant])


### Stratified Cross Validation



In [ ]:
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=40)

# Prepare arrays for storing metrics or other evaluation results
from sklearn.metrics import f1_score, precision_score, recall_score

f1_scores = []
precisions = []
recalls = []

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer
import random
import numpy as np
def set_seed(seed_value=42):
    """Set seed for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)  # if you are using GPU
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


In [ ]:
set_seed(42)  #

### BERTurk Cross Validation

In [ ]:
import numpy as np
from sklearn.metrics import f1_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from torch.utils.data import Dataset
import torch
from sklearn.metrics import classification_report


# Define the compute_metrics function for F1 score
def compute_metrics(p):
    predictions = np.argmax(p.predictions, axis=1)
    return {"f1": f1_score(p.label_ids, predictions, average='binary')}  # You can change average to 'binary' if it's binary classification

# Load the tokenizer and model specifically for Turkish tweets
model_name = "dbmdz/bert-base-turkish-128k-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)



# Define a dataset class
class TweetDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Function to tokenize data
def tokenize_data(X, tokenizer):
    return tokenizer(X.tolist(), padding=True, truncation=True, max_length=128, return_tensors='pt')  # Adjusted max_length for tweets

# Tokenize the data
for train_index, val_index in skf.split(df_combined['Tweet'], df_combined['label']):
    # Split the data into train and validation for the current fold
    X_train, X_val = df_combined.iloc[train_index]['Tweet'], df_combined.iloc[val_index]['Tweet']
    y_train, y_val = df_combined.iloc[train_index]['label'], df_combined.iloc[val_index]['label']

    train_encodings = tokenize_data(X_train, tokenizer)
    val_encodings = tokenize_data(X_val, tokenizer)

    # Create datasets
    train_dataset = TweetDataset(train_encodings, y_train.tolist())
    val_dataset = TweetDataset(val_encodings, y_val.tolist())

    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Adjust num_labels as per your task


# Set up training arguments
    training_args = TrainingArguments(
      output_dir='./results',            # Output directory
      evaluation_strategy="epoch",       # Evaluation is done at the end of each epoch
      save_strategy="epoch",             # Save model at the end of each epoch
      learning_rate=2e-5,                # Defines the learning rate
      per_device_train_batch_size=8,     # Batch size for training
      per_device_eval_batch_size=16,     # Batch size for evaluation
      num_train_epochs=3,                # Number of training epochs
      weight_decay=0.01,                 # Weight decay for regularization
      load_best_model_at_end=True,       # Load the best model when finished training
      metric_for_best_model='f1'         # Use F1 score to evaluate the best model
      )

# Initialize the Trainer
    trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=val_dataset,
      compute_metrics=compute_metrics  # Use F1 score for evaluation
  )

# Train the model
    trainer.train()

    eval_results = trainer.evaluate(val_dataset)
    f1_scores.append(eval_results['eval_f1'])
    print(f"Validation Results for Current Fold: {eval_results}")

    predictions = trainer.predict(val_dataset)
    pred_labels = np.argmax(predictions.predictions, axis=-1)
    print(classification_report(predictions.label_ids, pred_labels, target_names=['Irrelevant', 'Relevant']))


# Evaluate the model on the test set



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-128k-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Tra

Epoch,Training Loss,Validation Loss,F1
1,No log,0.271107,0.791667
2,No log,0.364546,0.836601
3,No log,0.251373,0.901408


<ipython-input-18-11912fc61d5e>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-18-11912fc61d5e>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-18-11912fc61d5e>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-18-11912fc61d5e>:27: UserWarning: To copy construct fro

Validation Results for Current Fold: {'eval_loss': 0.2513731122016907, 'eval_f1': 0.9014084507042253, 'eval_runtime': 73.948, 'eval_samples_per_second': 3.732, 'eval_steps_per_second': 0.243, 'epoch': 3.0}


<ipython-input-18-11912fc61d5e>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


...

Validation Results for Current Fold: {'eval_loss': 0.11319660395383835, 'eval_f1': 0.9496402877697843, 'eval_runtime': 98.7569, 'eval_samples_per_second': 2.795, 'eval_steps_per_second': 0.182, 'epoch': 3.0}


<ipython-input-18-11912fc61d5e>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


              precision    recall  f1-score   support

  Irrelevant       0.99      0.98      0.98       207
    Relevant       0.94      0.96      0.95        69

    accuracy                           0.97       276
   macro avg       0.96      0.97      0.97       276
weighted avg       0.97      0.97      0.97       276



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-128k-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-18-11912fc61d5e>:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss


In [ ]:
all_predictions = []
all_true_labels = []

In [ ]:
predictions = trainer.predict(val_dataset)
pred_labels = np.argmax(predictions.predictions, axis=-1)
all_predictions.extend(pred_labels)
all_true_labels.extend(predictions.label_ids)


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(all_true_labels, all_predictions, target_names=['Class 0', 'Class 1']))


              precision    recall  f1-score   support

     Class 0       0.98      0.98      0.98       182
     Class 1       0.87      0.87      0.87        30

    accuracy                           0.96       212
   macro avg       0.92      0.92      0.92       212
weighted avg       0.96      0.96      0.96       212



In [ ]:
f1_scores

[0.8923076923076922, 0.8571428571428571, 0.8666666666666667]

In [ ]:
from sklearn.metrics import classification_report
predictions = trainer.predict(test_dataset)
predicted_labels = predictions.predictions.argmax(-1)
print(classification_report(y_test, predicted_labels, target_names=['Irrelevant', 'Relevant']))

<ipython-input-14-2f56f71eed2e>:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


              precision    recall  f1-score   support

  Irrelevant       0.97      0.96      0.97       276
    Relevant       0.86      0.90      0.88        69

    accuracy                           0.95       345
   macro avg       0.92      0.93      0.92       345
weighted avg       0.95      0.95      0.95       345

